In [8]:
import numpy as np
import bordado as bd

## Gerar dado sintético

$g_z = \frac{Gm(z-z_c)}{((x-x_c)^2+(y-y_c)^2+(z-z_c)^2)}$

$x,y: 10km - 50km$

$z: 0$

$G = 6.6743×10−11$

In [ ]:
def model(x,y,z,x_c,y_c,z_c,m):


In [10]:
def jacobian(dx,dy,dz,dm):
    pass

In [ ]:
G = 6.6743e-11
region = [10e3, 50e3, 10e3, 50e3]
x, y = bd.random_coordinates(region, size=1000, random_seed=42)
z = np.zeros_like(x)
m = 1e11
x_c = 25e3
x_c = 25e3
z_c = -15e3

g